In [29]:
# Setup
import urllib3
import certifi
from bs4 import BeautifulSoup
import pandas as pd
import json

http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

In [30]:
# Define score functions
def pitcher_score(df):
    return (df['W'] * 20) + (df['SV'] * 10) + df['K'] + df['IP'] + (df['ER'] * -1) + (df['L'] * -5)


def hitter_score(df):
    return (df['HR'] * 4) + (df['3B'] * 3) + (df['2B'] * 2) + df['H'] + df['R'] + df['RBI'] + df['BB'] + df['SB']

In [51]:
# Pull Sportsnet Player Lists
base_url = "https://fantasy.sportsnet.ca/sportsnet/baseball18/player_stats"
group_param = "pick_stats_group"

scrape_pages = {
    "C/DH": 1,
    "1B/3B": 2,
    "2B/SS": 3,
    "OF": 4,
    "SP": 5,
    "RP": 6
}

player_frames = {}

for key, value in scrape_pages.items():

    response = http.request("GET", base_url + "?" + group_param + "=" + str(value))
    
    soup = BeautifulSoup(response.data, 'html.parser')
    table = soup.find('table', attrs={'id': 'playerstats'})
    header = [th.text for th in table.find('thead').select('th')]
    players = [[td.text.strip().split('\n')[0] for td in row.select('td')] for row in table.tbody.find_all('tr')]
    cols = zip(*players)
    tbl_d = {name: col for name, col in zip(header, cols)}
    df = pd.DataFrame(tbl_d, columns=header)
    name = df['Player'].str.split(', ')
    df["First"] = name.str[1]
    df['Last'] = name.str[0]
    df['Player'] = name.str[1] + " " + name.str[0]
    player_frames[key] = df.apply(pd.to_numeric, errors='ignore')


In [52]:
# Load Injury list from MLB
injury_url = "http://mlb.mlb.com/fantasylookup/json/named.wsfb_news_injury.bam"

response = http.request("GET", injury_url, headers={'Accept-Encoding': 'UTF-8'})
data = json.loads(response.data)
players = data['wsfb_news_injury']['queryResults']['row']
injuries = pd.DataFrame(players)
injuries['Player'] = injuries['name_first'] + " " + injuries['name_last']

In [53]:
# Load Fantasy Pro projections
hitters = pd.read_csv("data/FantasyPros_2018_wk2_Projections_H.csv")
hitters['Score'] = hitter_score(hitters)

pitchers = pd.read_csv("data/FantasyPros_2018_wk2_Projections_P.csv")
pitchers['Score'] = pitcher_score(pitchers)


In [67]:
# Join Sportsnet data with projections and Injury report
frames = {}

for key, value in player_frames.items():
    print(key + ":")
    merge_frame = hitters
    if key == "SP" or key == "RP":
        merge_frame = pitchers
    print("Original Size: " + str(len(value.index)))
    result = pd.merge(value, merge_frame, on='Player', suffixes=['_actual', '_projected'])
    print("Projection Size: " + str(len(result.index)))
    result = pd.merge(result, injuries, how='left', on='Player', suffixes=['', '_injuries'])
    result = result[result['injury_status'].isnull()]
    print("Injuries Size: " + str(len(result.index)))
    print("")
    frames[key] = result


C/DH:
Original Size: 86
Projection Size: 65
Injuries Size: 62

1B/3B:
Original Size: 72
Projection Size: 60
Injuries Size: 55

2B/SS:
Original Size: 87
Projection Size: 76
Injuries Size: 72

OF:
Original Size: 131
Projection Size: 106
Injuries Size: 95

SP:
Original Size: 174
Projection Size: 131
Injuries Size: 126

RP:
Original Size: 88
Projection Size: 65
Injuries Size: 63



In [ ]:
# Find Max expected performers in all player values for all player categories
max_players = {}

for key, frame in frames.items():
    idx = frame.groupby(['PV'])['Score'].transform(max) == frame['Score']
    max_players[key] = frame[idx]


Empty DataFrame
Columns: [PV, Player, Team_actual, GP, W_actual, K_actual, SV_actual, L_actual, IP_actual, ER_actual, PTS, First, Last, VBR, Team_projected, Positions, IP_projected, K_projected, W_projected, SV_projected, ERA, WHIP, ER_projected, H, BB, HR, G, GS, L_projected, CG, Points, Own, Score, display_ts, due_back, injury_desc, injury_status, injury_update, insert_ts, league_id, name_first, name_last, player_id, position, team_id, team_name]
Index: []

[0 rows x 46 columns]


In [41]:
# Build all possible max player teams into a dataframe
columns = ["C/DH", "1B/3B", "2B/SS", "OF1", "OF2", "SP1", "SP2", "RP", "PV Total", "xScore"]
data = []

for cdh_index, cdh in max_players["C/DH"].iterrows():
    for fbtb_index, fbtb in max_players["1B/3B"].iterrows():
        for sbss_index, sbss in max_players["2B/SS"].iterrows():
            for of1_index, of1 in max_players["OF"].iterrows():
                for of2_index, of2 in max_players["OF"].iterrows():
                    if of1["Player"] == of2["Player"]:
                        continue
                    if of2['PV'] + of1['PV'] + sbss['PV'] + fbtb['PV'] + cdh['PV'] >= 18:
                        continue
                    for sp1_index, sp1 in max_players["SP"].iterrows():
                        if sp1['PV'] + of2['PV'] + of1['PV'] + sbss['PV'] + fbtb['PV'] + cdh['PV'] >= 19:
                            continue
                        for sp2_index, sp2 in max_players["SP"].iterrows():
                            if sp1["Player"] == sp2["Player"]:
                                continue
                            if sp2['PV'] + sp1['PV'] + of2['PV'] + of1['PV'] + sbss['PV'] + fbtb['PV'] + cdh['PV'] >= 20:
                                continue
                            for rp_index, rp in max_players["RP"].iterrows():
                                if rp['PV'] + sp2['PV'] + sp1['PV'] + of2['PV'] + of1['PV'] + sbss['PV'] + fbtb['PV'] + cdh['PV'] > 20:
                                    continue
                                data.append([cdh['Player'], fbtb['Player'], sbss['Player'], of1['Player'],
                                             of2['Player'], sp1['Player'], sp2['Player'], rp['Player'],
                                             rp['PV'] + sp1['PV'] + sp2['PV'] + of2['PV'] + of1['PV'] + sbss['PV'] 
                                             + fbtb['PV'] + cdh['PV'],
                                             rp['Score'] + sp1['Score'] + sp2['Score'] + of2['Score'] + of1['Score']
                                             + sbss['Score'] + fbtb['Score'] + cdh['Score']])

cols = zip(*data)
tbl_d = {name: col for name, col in zip(columns, cols)}
teams = pd.DataFrame(tbl_d, columns=columns)

In [42]:
# Analyze Teams
teams.sort_values('xScore', ascending=False).head(25)

,C/DH,1B/3B,2B/SS,OF1,OF2,SP1,SP2,RP,PV Total,xScore
20841,Miguel Montero,Nolan Arenado,Scooter Gennett,Bryce Harper,Lewis Brinson,Masahiro Tanaka,Jacob deGrom,Brad Hand,20,228.30
20832,Miguel Montero,Nolan Arenado,Scooter Gennett,Bryce Harper,Lewis Brinson,Jacob deGrom,Masahiro Tanaka,Brad Hand,20,228.30
21000,Miguel Montero,Nolan Arenado,Scooter Gennett,Lewis Brinson,Bryce Harper,Jacob deGrom,Masahiro Tanaka,Brad Hand,20,228.30
21009,Miguel Montero,Nolan Arenado,Scooter Gennett,Lewis Brinson,Bryce Harper,Masahiro Tanaka,Jacob deGrom,Brad Hand,20,228.30
1885,Shin-Soo Choo,Jose Ramirez,Scooter Gennett,Lewis Brinson,Bryce Harper,Masahiro Tanaka,Jacob deGrom,Brad Hand,20,228.07
1708,Shin-Soo Choo,Jose Ramirez,Scooter Gennett,Bryce Harper,Lewis Brinson,Jacob deGrom,Masahiro Tanaka,Brad Hand,20,228.07
1717,Shin-Soo Choo,Jose Ramirez,Scooter Gennett,Bryce Harper,Lewis Brinson,Masahiro Tanaka,Jacob deGrom,Brad Hand,20,228.07
1876,Shin-Soo Choo,Jose Ramirez,Scooter Gennett,Lewis Brinson,Bryce Harper,Jacob deGrom,Masahiro Tanaka,Brad Hand,20,228.07
22605,Miguel Montero,Jose Ramirez,Scooter Gennett,Bryce Harper,Jesse Winker,Jacob deGrom,Masahiro Tanaka,Raisel Iglesias,20,227.34
22618,Miguel Montero,Jose Ramirez,Scooter Gennett,Bryce Harper,Jesse Winker,Masahiro Tanaka,Jacob deGrom,Raisel Iglesias,20,227.34
